# 공공데이터를 활용한 미세먼지 농도 예측 (2)

### 목표
* 공공 데이터를 기반으로 머신러닝을 통해 미세먼지의 농도 예측

### 데이터
* 1 air_2021.csv : 2021년 미세먼지 데이터  
* 2 air_2022.csv : 2022년 미세먼지 데이터  
* 3 weather_2021.csv : 2021년 날씨 데이터  
* 4 weather_2022.csv : 2022년 날씨 데이터    
<br>
* Train 데이터 : 2021년의 미세먼지 농도 데이터
* Test 데이터 : 2022년의 미세먼지 농도 데이터
<br><br>
출처: 에어코리아
---

In [40]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score

from tqdm import tqdm

In [4]:
x_train = pd.read_csv("data/x_train.csv")
y_train = pd.read_csv("data/y_train.csv")
x_test = pd.read_csv("data/x_test.csv")
y_test = pd.read_csv("data/y_test.csv")

## 5> 머신러닝 모델링
* **LienarRegression** : 선형회귀, 실제 값과 근사 값 간 오차의 제곱의 합이 최소가 되는 선형 모델
* **KNeighborsRegressor** : K-최근접 이웃 회귀, 주변의 가장 가까운 K개의 샘플을 참고하는 모델
* **RandomForestRegressor** : 이진 분류를 사용하는 결정 트리 여러 개를 학습하여 그 예측을 종합하는 앙상블 모델
* **GradientBoostingRegressor** : 여러 개의 결정 트리를 순차적으로 사용해 잔차를 최소화하는 앙상블 모델
<br><br>
* 평가 : MSE, R-squared Score

### 5-1> LinearRegression
* **fit_intercept**
    * 모델에 절편항의 포함 여부
    - **True**: 모델 예측값에 상수항을 추가, 회귀선이 반드시 원점을 지나가지는 않음, 예측력 향상, 과적합 가능
    - **False**: 모델 예측값에 상수항을 추가하지 않음, 회귀선이 반드시 원점을 지나감, 모델 단순화, 일반화 성능 저하 가능

In [ ]:
# 모델 튜닝
param_lr = {'fit_intercept': [True, False]}

model_lr = GridSearchCV(LinearRegression(),
                        param_grid=param_lr,
                        cv=5, # cv=KFold(n_splits=5, shuffle=True, random_state=2)
                        scoring='r2')

In [7]:
# 학습
model_lr.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False]}, scoring='r2')

In [8]:
# 예측
y_pred_lr = model_lr.predict(x_test)

In [17]:
model_lr.best_estimator_

LinearRegression()

In [9]:
def model_evaluation(y_pred, y_test=y_test):
    MSE = np.round(mean_squared_error(y_test, y_pred), 3)
    RMSE = np.round(root_mean_squared_error(y_test, y_pred), 3)
    R2 = np.round(r2_score(y_test, y_pred), 3)

    print("MSE:", MSE)
    print("RMSE:", RMSE)
    print("R2:", R2)

    return MSE, RMSE, R2

In [12]:
# 평가
lr_mse, lr_rmse, lr_r2 = model_evaluation(y_pred_lr)
# print(model_lr.score(x_test, y_test)) ## r2_score == model.score

MSE: 37.033
RMSE: 6.085
R2: 0.933


### 5-2> KNeighborsRegressor
* **n_neighbors**
    * 모델에 사용할 가장 가까운 이웃의 수, 홀수로 해야 과반수 비교를 할 수 있음
* **weights**
    * 이웃에 가중치 부여하는 방법
    - **uniform**: 모든 이웃에 동일한 가중치 부여
    - **distance**: 가까운 거리의 이웃일수록 더 큰 가중치 부여
* **metric**
    * 거리 측정 방법
    - **euclidean**: 유클리드 거리, $\sqrt{\displaystyle\sum_{i=1}^{n} (p_i-q_i)^2} $
    - **manhattan**: 맨해튼 거리, $\displaystyle\sum_{i=1}^{n} |p_i-q_i| $

In [13]:
# 모델 튜닝
param_kn = {'n_neighbors': [3, 5, 7, 9, 11],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']}

model_kn = GridSearchCV(KNeighborsRegressor(),
                        param_grid=param_kn,
                        cv=5,
                        scoring='r2')

In [14]:
# 학습
model_kn.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 9, 11],
                         'weights': ['uniform', 'distance']},
             scoring='r2')

In [15]:
# 예측
y_pred_kn = model_kn.predict(x_test)

In [16]:
model_kn.best_estimator_

KNeighborsRegressor(metric='euclidean', n_neighbors=9, weights='distance')

In [18]:
# 평가
kn_mse, kn_rmse, kn_r2 = model_evaluation(y_pred_kn)

MSE: 66.504
RMSE: 8.155
R2: 0.88


### 5-3> RandomForestRegressor
* **n_estimators**
    * 랜덤 포레스트를 구성하는 결정 트리의 개수, 많을수록 모델 성능 향상되지만 계산 비용 증가
* **max_depth**
    * 각 결정 트리의 최대 깊이, 깊을수록 과적합 가능성 증가
* **min_samples_split**
    * 결정 트리에서 노드를 분할하기 위한 최소 샘플의 수, ex) split=5 일때, 어떤 노드의 샘플 수가 6이면 5이하가 되도록 해당 노드를 분할

In [53]:
# 모델 튜닝
# param_rf = {'n_estimators': [100, 200, 300],
#             'max_depth': range(3, 12, 2), # 3, 5, 7, 9, 11
#             'min_samples_split': [2, 5, 10]}
param_rf = {'max_depth': [3, 5, 7],
            'min_samples_split': [2, 10]} # 너무 오래걸려서 일부 생략

model_rf = GridSearchCV(RandomForestRegressor(random_state=2),
                        param_grid=param_rf,
                        cv=5,
                        scoring='r2')

In [41]:
# 학습 준비
print(y_train.shape)

y_train_1d = y_train.to_numpy().ravel() # 1차원 배열로 변환
y_test_1d = y_test.to_numpy().ravel()

(8711, 1)


In [54]:
# 학습
for i in tqdm(range(5 * len(model_rf.param_grid))):
    model_rf.fit(x_train, y_train_1d)
# 10m 9.9s

100%|██████████| 10/10 [10:09<00:00, 60.99s/it]


In [75]:
model_rf.best_estimator_

RandomForestRegressor(max_depth=7, min_samples_split=10, random_state=2)

In [55]:
# 예측
y_pred_rf = model_rf.predict(x_test)

In [56]:
# 평가
rf_mse, rf_rmse, rf_r2 = model_evaluation(y_pred_rf, y_test=y_test_1d)

MSE: 38.827
RMSE: 6.231
R2: 0.93


### 5-4> GradientBoostingRegressor
* **learning_rate**
    * 학습 모델의 에러에 대한 가중치를 수정하는 기준, 최적의 값을 찾는 탐색 범위
    * GB에서는 각 결정 트리가 전체 예측에 기여하는 정도를 조절, 작을수록 안정적으로 학습, 클수록 학습 비용 적으나 과적합 가능

In [76]:
# 모델 튜닝
param_gb = {'learning_rate': [0.1, 0.01],
            'max_depth': [3, 5]}

model_gb = GridSearchCV(GradientBoostingRegressor(random_state=2),
                        param_grid=param_gb,
                        cv=5,
                        scoring='r2')

In [77]:
# 학습
for i in tqdm(range(5 * len(model_gb.param_grid))):
    model_gb.fit(x_train, y_train_1d)

100%|██████████| 10/10 [07:44<00:00, 46.47s/it]


In [78]:
# 예측
y_pred_gb = model_gb.predict(x_test)

In [79]:
# 평가
gb_mse, gb_rmse, gb_r2 = model_evaluation(y_pred_gb)

MSE: 37.803
RMSE: 6.148
R2: 0.932


### 5-5> 모델 비교

In [89]:
df_eval = pd.DataFrame({'mse':[lr_mse, kn_mse, rf_mse, gb_mse],
                        'rmse':[lr_rmse, kn_rmse, rf_rmse, gb_rmse],
                        'r2':[lr_r2, kn_r2, rf_r2, gb_r2]},
                        index=['LinearRegression', 'KNeighborsRegressor', 'RandomForestRegressor', 'GradientBoostingRegressor'])

df_eval

,mse,rmse,r2
LinearRegression,37.033,6.085,0.933
KNeighborsRegressor,66.504,8.155,0.880
RandomForestRegressor,38.827,6.231,0.930
GradientBoostingRegressor,37.803,6.148,0.932


In [83]:
# rmse가 낮고 r2가 높은 모델 선택택
best_model = model_lr.best_estimator_
best_model

LinearRegression()

## 6> 추가) 데이터 선택 후 재 모델링
* 앞선 인사이트에서 전체 데이터보다 특정 기간(여름)의 데이터에 대해 상관성이 높아지는 것을 확인했다.
* 이를 참고하여, **test 데이터가 1월부터 3월까지이므로 해당 기간의 데이터만 사용할 때의 정확도를 다시 확인해보자.**

In [71]:
Air_21 = pd.read_csv("data/Air_21_prep.csv")
Air_22 = pd.read_csv("data/Air_22_prep.csv")

len22 = len(Air_22)
display(Air_21.iloc[len22-2:len22, :])
display(Air_22.tail(2))

,측정일시,SO2,CO,O3,NO2,PM10,PM25
2158,2021-03-31 22:00:00,0.003,0.5,0.023,0.035,39.0,18.0
2159,2021-03-31 23:00:00,0.003,0.7,0.004,0.053,35.0,17.0


,측정일시,SO2,CO,O3,NO2,PM10,PM25
2158,2022-03-31 22:00:00,0.003,0.3,0.036,0.015,11.0,5.0
2159,2022-03-31 23:00:00,0.002,0.4,0.033,0.017,9.0,5.0


In [70]:
test_len = len(x_test)
x_train2 = x_train.iloc[:test_len, :]
y_train2 = y_train.iloc[:test_len, :]

In [84]:
best_model.fit(x_train2, y_train2)

LinearRegression()

In [85]:
y_pred2 = best_model.predict(x_test)

In [86]:
y2_mse, y2_rmse, y2_r2 = model_evaluation(y_pred2)

MSE: 40.447
RMSE: 6.36
R2: 0.927


In [94]:
new_row = pd.Series({'mse':y2_mse, 'rmse':y2_rmse, 'r2':y2_r2}, name="LR_DataSplit")
df_eval2 = pd.concat([df_eval, new_row.to_frame().T])
df_eval2

,mse,rmse,r2
LinearRegression,37.033,6.085,0.933
KNeighborsRegressor,66.504,8.155,0.880
RandomForestRegressor,38.827,6.231,0.930
GradientBoostingRegressor,37.803,6.148,0.932
LR_DataSplit,40.447,6.360,0.927


### 6-1> 결론
* 시계열 데이터일지라도
<br>4배 많은 원본 데이터를 사용하여 학습하는 것이
<br>기간을 맞춘 데이터만 학습하는 것보다 모델의 성능이 조금 더 좋다.
